## Parte 2

Neste exercício você irá preparar uma base de dados de imagens de expressões faciais e treinar uma regressão logística nesses dados. Para isso usaremos uma base de dados de um desafio [publicado no site Kaggle em 2013](https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge), e descrito [neste artigo do arXiv.org](http://arxiv.org/abs/1307.0414). Esta base consiste em um arquivo [em formato CSV](https://pt.wikipedia.org/wiki/Comma-separated_values), onde constam milhares de imagens monocromáticas de 48x48 pixels de rostos de diferentes pessoas. Para cada imagem é dado um número inteiro de 0 a 6 que representa uma expressão seguindo esta legenda: 0=Raiva, 1=Nojo, 2=Medo, 3=Felicidade, 4=Tristeza, 5=Surpresa, 6=Neutro.

Nesta base de dados há exatamente 28709 imagens destinadas para treinamento, 3589 destinadas para validação e 3589 para teste (respectivamente marcadas como 'Training', 'PublicTest' e 'PrivateTest').

A regressão logística deverá ser implementada utilizando a biblioteca TensorFlow, seguindo os mesmos moldes do exercício "Assignment 1: notMNIST" do [curso de Deep Learning da Udacity](https://classroom.udacity.com/courses/ud730).

Critérios de avaliação:
- 1) Abrir o arquivo CSV fornecido e acessar os dados a partir do código em Python (0,5 ponto)
- 2) Separar adequadamente dados de treinamento, validação e teste (0,5 ponto)
- 3) Criar arrays do NumPy com as devidas dimensões e tipagem, e preenchê-las com os dados lidos do arquivo (1,0 ponto)
- 4) Armazenar as no formato one-hot e as normalizar as imagens (1,0 ponto)
- 5) Criar um arquivo binário do Pickle com todos os dados já formatados em arrays do NumPy (1,0 ponto)
- 6) Ler o arquivo binário do Pickle, restaurando todos os dados já formatados em arrays do NumPy (1,0 ponto)
- 7) Mostrar 10 exemplos aleatórios de imagens, imprimindo suas classes correspondentes (1,0 ponto)
- 8) Implementar o Grafo da Regressão Linear no TensorFlow com as constantes, os placeholders e as variáveis, realizando as dimensões e operações corretas, incluindo erro apropriado para *one-hot encoding* (1,0 ponto)
- 9) Implementar a Sessão no TensorFlow, fazendo o treinamento por lotes, e mostrando o erro de treinamento diminuindo (1,0 ponto)
- 10) Mostrar o erro nos dados de validação a cada época do treinamento (1,0 ponto)
- 11) Mostrar o erro nos dados de teste ao final de todo treinamento (1,0 ponto)
- **Ponto Extra:** Implementar e treinar uma rede neural do tipo perceptron com duas camadas ou mais (1,0 ponto)

In [ ]:
# CONSIDERANDO A EXECUÇÃO SEPARADAMENTE DA PARTE 1
# Importa as bibliotecas que serão utilizadas 
from __future__ import print_function
import csv
import numpy as np
from matplotlib import pyplot as plt
from six.moves import cPickle as pickle
import os
import tensorflow as tf
import random
from sklearn import datasets
from sklearn.model_selection import train_test_split
from six.moves import range
# Configura precisão para impressão de arrays do NumPy
np.set_printoptions(formatter={'float':lambda x: '%+01.2f ' % x})

In [ ]:
# Define dicionário para as etiquetas que
# representam cada classe
image_size = 48
num_labels = 7
label_dict = {0:'Raiva', 1:'Nojo', 2:'Medo', 3:'Felicidade', 4:'Tristeza', 5:'Surpresa', 6:'Neutro' }

use_dict = {1:'Training' , 2:'PublicTest' , 3:'PrivateTest'}

# Define função auxiliar para retornar o nome
# da classe correspondente para cada índice

def get_label(label_array):
    return label_dict[np.argmax(label_array)]

def get_use(use_array):
    return use_dict[use_array]

In [ ]:
# Define função auxiliar para leitura dos
# dados a partir de arquivos no formato CSV

def readfile(filename, total, dados):
    with open(filename,'r') as csvfile:
        rows = csv.reader(csvfile)
        i = 0
        
        num1 = 0 # Treino
        num2 = 0 # Validação
        num3 = 0 # Teste
        
        for row in rows:
            if i > 0:
                image = np.zeros((total,48*48),dtype=np.float32)
                
                label = np.zeros((total,7),dtype=np.float32)
                label = (np.arange(0,7) == np.array(row[0]).astype(np.float32)) # one-hot
                
                data = np.array(row[1].split(' '))
                data = data.astype(np.float32)
                
                image = (data / 255.0) - 0.5
                
                if row[2] == get_use(1):
                    dados.train_labels[num1,:] = (label).astype(np.float32)
                    dados.train_images[num1,:] = (image)
                    num1 = num1+1
                
                elif row[2] == get_use(2):
                    dados.valid_labels[num2,:] = (label).astype(np.float32)
                    dados.valid_images[num2,:] = (image)
                    num2 = num2+1
                
                else:
                    dados.test_labels[num3,:] = (label)
                    dados.test_images[num3,:] = (image)
                    num3 = num3+1
            
            i = i + 1
            print(i)

In [ ]:
class Dados:
    
    def __init__(self):
        self.total = (28709+3589+3589)
        self.train_images = np.zeros((28709,image_size*image_size),dtype=np.float32)
        self.train_labels = np.zeros((28709,num_labels),dtype=np.float32)

        self.test_images = np.zeros((3589,image_size*image_size),dtype=np.float32)
        self.test_labels = np.zeros((3589,num_labels),dtype=np.float32)

        self.valid_images = np.zeros((3589,image_size*image_size),dtype=np.float32)
        self.valid_labels = np.zeros((3589,num_labels),dtype=np.float32)
    
    def ReadFil(self):
        #preenche os arrays
        readfile('fer2013.csv',self.total,self)
        
    def getTrain(self):
        return self.train_images,self.train_labels
    
    def getTest(self):
        return self.test_images,self.test_labels
    
    def getValid(self):
        return self.valid_images,self.valid_labels
    

In [ ]:
data_root = '.' 
file_name = 'DadosQuestao2.pickle'

def SavePickleDados(dados):
    pickle_file = os.path.join(data_root, file_name)

    try:
        f = open(pickle_file, 'wb')
        save = { 'Dados': dados }
        pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
        f.close()
    
    except Exception as e:
        print('Unable to save data to', pickle_file, ':', e)
        raise

def LoadPickleDados():
    pik = pickle.load(open(file_name, 'rb'))
    return pik['Dados']

In [ ]:
#carrega dados 
bd = Dados()
bd.ReadFil()
SavePickleDados(bd)
dados = LoadPickleDados()

In [ ]:
test_images, test_labels = dados.getTest()
train_images, train_labels = dados.getTrain()
valid_images, valid_labels = dados.getValid()

In [ ]:
# Mostra 10 exemplos
for j in range(10):
    i = np.random.randint(0,test_images.shape[0])
    image = test_images[i,:].reshape((image_size,image_size))
    print(i)
    print(get_label(test_labels[i,:]))
    plt.imshow(image)
    plt.show()

In [ ]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_images)
    tf_test_dataset = tf.constant(test_images)
  
    # Variables.
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) # Regressão Lin.
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss) # eta/alfa/taxa_correcao
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [ ]:
# Porcentagem de acerto
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])


In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (test_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_images[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))